# Part 2: Intro to Federated Learning

In the last section, we learned about PointerTensors, which create the underlying infrastructure we need for privacy preserving Deep Learning. In this section, we're going to see how to use these basic tools to implement our first privacy preserving deep learning algorithm, Federated Learning.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

### What is Federated Learning?

It's a simple, powerful way to train Deep Learning models. If you think about training data, it's always the result of some sort of collection process. People (via devices) generate data by recording events in the real world. Normally, this data is aggregated to a single, central location so that you can train a machine learning model. Federated Learning turns this on its head!

Instead of bringing training data to the model (a central server), you bring the model to the training data (wherever it may live).

The idea is that this allows whoever is creating the data to own the only permanent copy, and thus maintain control over who ever has access to it. Pretty cool, eh?

# Section 2.1 - A Toy Federated Learning Example

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

Note: If this API is un-familiar to you - head on over to [fast.ai](http://fast.ai) and take their course before continuing in this tutorial.

In [0]:
import torch
from torch import nn
from torch import optim

In [0]:

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [7]:
train()

tensor(1.7879)
tensor(0.6441)
tensor(0.4126)
tensor(0.2932)
tensor(0.2121)
tensor(0.1546)
tensor(0.1134)
tensor(0.0836)
tensor(0.0620)
tensor(0.0461)
tensor(0.0344)
tensor(0.0258)
tensor(0.0194)
tensor(0.0146)
tensor(0.0111)
tensor(0.0084)
tensor(0.0063)
tensor(0.0048)
tensor(0.0037)
tensor(0.0028)


And there you have it! We've trained a basic model in the conventional manner. All our data is aggregated into our local machine and we can use it to make updates to our model. Federated Learning, however, doesn't work this way. So, let's modify this example to do it the Federated Learning way! 

So, what do we need:

- create a couple workers
- get pointers to training data on each worker
- updated training logic to do federated learning

    New Training Steps:
    - send model to correct worker
    - train on the data located there
    - get the model back and repeat with next worker

In [8]:

!pip install syft

     |████████████████████████████████| 184kB 5.0MB/s 
     |████████████████████████████████| 204kB 46.3MB/s 
     |████████████████████████████████| 71kB 26.7MB/s 
     |████████████████████████████████| 450kB 54.8MB/s 
     |████████████████████████████████| 389kB 35.5MB/s 
     |████████████████████████████████| 1.4MB 55.0MB/s 
     |████████████████████████████████| 51kB 15.5MB/s 
     |████████████████████████████████| 276kB 53.6MB/s 
     |████████████████████████████████| 122kB 47.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [0]:
import syft as sy
hook = sy.TorchHook(torch)


In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [13]:
train()

tensor(1.6169, requires_grad=True)
tensor(3.8155, requires_grad=True)
tensor(0.1905, requires_grad=True)
tensor(0.1517, requires_grad=True)
tensor(0.1771, requires_grad=True)
tensor(0.0612, requires_grad=True)
tensor(0.1161, requires_grad=True)
tensor(0.0379, requires_grad=True)
tensor(0.0754, requires_grad=True)
tensor(0.0247, requires_grad=True)
tensor(0.0495, requires_grad=True)
tensor(0.0165, requires_grad=True)
tensor(0.0329, requires_grad=True)
tensor(0.0112, requires_grad=True)
tensor(0.0221, requires_grad=True)
tensor(0.0077, requires_grad=True)
tensor(0.0151, requires_grad=True)
tensor(0.0054, requires_grad=True)
tensor(0.0104, requires_grad=True)
tensor(0.0039, requires_grad=True)


## Well Done!

And voilà! We now are training a very simple Deep Learning model using Federated Learning! We send the model to each worker, generate a new gradient, and then bring the gradient back to our local server where we update our global model. Never in this process do we ever see or request access to the underlying training data! We preserve the privacy of Bob and Alice!!!

## Shortcomings of this Example

So, while this example is a nice introduction to Federated Learning, it still has some major shortcomings. Most notably, when we call `model.get()` and receive the updated model from Bob or Alice, we can actually learn a lot about Bob and Alice's training data by looking at their gradients. In some cases, we can restore their training data perfectly! 

So, what is there to do? Well, the first strategy people employ is to **average the gradient across multiple individuals before uploading it to the central server**. This strategy, however, will require some more sophisticated use of PointerTenor objects. So, in the next section, we're going to take some time to learn about more advanced pointer functionality and then we'll upgrade this Federated Learning example


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)